<h1>Multi Model Server Container for Differential Deep Learning</h1>

THE NOTEBOOK DOESN'T WORK WITH GPU (due to MMS)!!!

Jupyter install
sudo /opt/conda/bin/conda install ipython jupyter 
/opt/conda/bin/jupyter notebook --ip=0.0.0.0 --port=8080 --no-browser

This notebook demonstrates how to build and use a custom Docker container for serving with Amazon SageMaker that leverages on the <strong>Multi Model Server (MMS)</strong> and <strong>sagemaker-inference-toolkit</strong> libraries for serving models through Amazon SageMaker's endpoints.
We will also see how MMS allows deploying multiple models on a single endpoint thanks to the multi-model endpoints functionality of Amazon SageMaker Hosting (https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html).

Useful links:
- https://github.com/awslabs/multi-model-server/
- https://github.com/aws/sagemaker-inference-toolkit

We start by defining some variables like the current execution role, the ECR repository that we are going to use for pushing the custom Docker container and a default Amazon S3 bucket to be used by Amazon SageMaker.

The whole deployment takes 10-12min.

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

isGPU = True
ecr_namespace = 'sagemaker-serving-containers/'
prefix = 'diffdl-container'

ecr_repository_name = ecr_namespace + prefix + ('-gpu' if isGPU else '')
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)
print(ecr_repository_name)

785577973223
us-east-1
arn:aws:iam::785577973223:role/admin
sagemaker-us-east-1-785577973223
sagemaker-serving-containers/diffdl-container-gpu


<h2>Build and push the container</h2>
We are now ready to build this container and push it to Amazon ECR. This task is executed using a shell script stored in the ../script/ folder. Let's take a look at this script and then execute it.

The script builds the Docker container, then creates the repository if it does not exist, and finally pushes the container to the ECR repository. The build task requires a few minutes to be executed the first time, then Docker caches build outputs to be reused for the subsequent build operations.

In [2]:
! ../scripts/build_and_push.sh $account_id $region $ecr_repository_name $isGPU

running sdist
running egg_info
writing src/multi_model_serving.egg-info/PKG-INFO
writing top-level names to src/multi_model_serving.egg-info/top_level.txt
writing dependency_links to src/multi_model_serving.egg-info/dependency_links.txt
reading manifest file 'src/multi_model_serving.egg-info/SOURCES.txt'
writing manifest file 'src/multi_model_serving.egg-info/SOURCES.txt'

running check

creating multi_model_serving-1.0.0
creating multi_model_serving-1.0.0/src
creating multi_model_serving-1.0.0/src/multi_model_serving
creating multi_model_serving-1.0.0/src/multi_model_serving.egg-info
copying files to multi_model_serving-1.0.0...
copying setup.py -> multi_model_serving-1.0.0
copying src/multi_model_serving/__init__.py -> multi_model_serving-1.0.0/src/multi_model_serving
copying src/multi_model_serving/blackscholes.py -> multi_model_serving-1.0.0/src/multi_model_serving
copying src/multi_model_serving/default_inference_handler.py -> multi_model_serving-1.0.0/src/multi_model_serving
copy

Get:21 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 liblcms2-2 amd64 2.9-1ubuntu0.1 [139 kB]
Get:22 http://archive.ubuntu.com/ubuntu bionic/main amd64 fonts-dejavu-core all 2.37-1 [1041 kB]
Get:23 http://archive.ubuntu.com/ubuntu bionic/main amd64 fontconfig-config all 2.12.6-0ubuntu2 [55.8 kB]
Get:24 http://archive.ubuntu.com/ubuntu bionic/main amd64 libfontconfig1 amd64 2.12.6-0ubuntu2 [137 kB]
Get:25 http://archive.ubuntu.com/ubuntu bionic/main amd64 libnspr4 amd64 2:4.18-1ubuntu1 [112 kB]
Get:26 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libnss3 amd64 2:3.35-2ubuntu2.12 [1220 kB]
Get:27 http://archive.ubuntu.com/ubuntu bionic/main amd64 libpcsclite1 amd64 1.8.23-1 [21.3 kB]
Get:28 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxi6 amd64 2:1.7.9-1 [29.2 kB]
Get:29 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxrender1 amd64 1:0.9.10-1 [18.7 kB]
Get:30 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 x11-common all 1:7.7+19ubuntu

Selecting previously unselected package liberror-perl.
Preparing to unpack .../34-liberror-perl_0.17025-1_all.deb ...
Unpacking liberror-perl (0.17025-1) ...
Selecting previously unselected package git-man.
Preparing to unpack .../35-git-man_1%3a2.17.1-1ubuntu0.8_all.deb ...
Unpacking git-man (1:2.17.1-1ubuntu0.8) ...
Selecting previously unselected package git.
Preparing to unpack .../36-git_1%3a2.17.1-1ubuntu0.8_amd64.deb ...
Unpacking git (1:2.17.1-1ubuntu0.8) ...
Selecting previously unselected package libatlas3-base:amd64.
Preparing to unpack .../37-libatlas3-base_3.10.3-5_amd64.deb ...
Unpacking libatlas3-base:amd64 (3.10.3-5) ...
Selecting previously unselected package libatlas-base-dev:amd64.
Preparing to unpack .../38-libatlas-base-dev_3.10.3-5_amd64.deb ...
Unpacking libatlas-base-dev:amd64 (3.10.3-5) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../39-openjdk-8-jdk-headless_8u292-b10-0ubuntu1~18.04_amd64.deb ...
Unpacking open

Adding debian:GlobalSign_Root_CA_-_R3.pem
Adding debian:GlobalSign_Root_CA_-_R6.pem
Adding debian:Global_Chambersign_Root_-_2008.pem
Adding debian:Go_Daddy_Class_2_CA.pem
Adding debian:Go_Daddy_Root_Certificate_Authority_-_G2.pem
Adding debian:Hellenic_Academic_and_Research_Institutions_ECC_RootCA_2015.pem
Adding debian:Hellenic_Academic_and_Research_Institutions_RootCA_2011.pem
Adding debian:Hellenic_Academic_and_Research_Institutions_RootCA_2015.pem
Adding debian:Hongkong_Post_Root_CA_1.pem
Adding debian:Hongkong_Post_Root_CA_3.pem
Adding debian:ISRG_Root_X1.pem
Adding debian:IdenTrust_Commercial_Root_CA_1.pem
Adding debian:IdenTrust_Public_Sector_Root_CA_1.pem
Adding debian:Izenpe.com.pem
Adding debian:Microsec_e-Szigno_Root_CA_2009.pem
Adding debian:Microsoft_ECC_Root_Certificate_Authority_2017.pem
Adding debian:Microsoft_RSA_Root_Certificate_Authority_2017.pem
Adding debian:NAVER_Global_Root_Certification_Authority.pem
Adding debian:NetLock_Arany_=Class_Gold=_Főtanúsítvány.pem
Add

      Successfully uninstalled pip-20.2.4
Removing intermediate container 9a6a20c507a8
 ---> cbbc567be9b9
Step 9/15 : RUN pip --no-cache-dir install multi-model-server                                sagemaker-inference                                retrying
 ---> Running in 3544eb7190b6
  Created wheel for sagemaker-inference: filename=sagemaker_inference-1.5.6-py2.py3-none-any.whl size=27006 sha256=4686028c1cd274633f90ab4bb5d564fe2896567111692793477faf426b5d7b2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vssfivy1/wheels/a2/da/43/c2da42a0388e208d1aaaec54b68b85a60df2a62e4d2a8c1993
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11421 sha256=19ff5477dda116a6447b309b7685780989a85dc94d5ebdc5916f561d0b15848f
  Stored in directory: /tmp/pip-ephem-wheel-cache-vssfivy1/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=230bd4754039008200fb18117a25fd8903746e

Removing intermediate container 8743f84422fb
 ---> 1e0578ca31e6
Step 11/15 : WORKDIR /
 ---> Running in 209272e7329e
Removing intermediate container 209272e7329e
 ---> cfb7ac37156b
Step 12/15 : COPY code/multi_model_serving-1.0.0.tar.gz /multi_model_serving-1.0.0.tar.gz
 ---> 1eab9b266885
Step 13/15 : RUN pip install /multi_model_serving-1.0.0.tar.gz && rm /multi_model_serving-1.0.0.tar.gz
 ---> Running in 89406753fa9f
Processing /multi_model_serving-1.0.0.tar.gz
  Created wheel for multi-model-serving: filename=multi_model_serving-1.0.0-py3-none-any.whl size=11955 sha256=20d591ec3ab5a306a4b4fd5625772a6560048705f2b93ebb5211f46b2e5b29f9
  Stored in directory: /root/.cache/pip/wheels/2d/aa/39/f055dee95e1415740e88e73ca29d1029a4107090b94d578aff
Successfully built multi-model-serving
Removing intermediate container 89406753fa9f
 ---> 1d720db356f7
Step 14/15 : COPY code/serve.py /serve.py
 ---> b1b19dd9f7a2
Step 15/15 : ENTRYPOINT ["python", "serve.py"]
 ---> Running in ef18be73b11b
Removing

<h2>Deploy with Amazon SageMaker</h2>


<h3>Get the container URI</h3>
Once we have correctly pushed our container to Amazon ECR, we are ready to deploy with Amazon SageMaker, which requires the ECR path to the Docker container used for serving as parameter for deployment.

In [4]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

785577973223.dkr.ecr.us-east-1.amazonaws.com/sagemaker-serving-containers/diffdl-container-gpu:latest


Test the image locally `docker run -it -p 3030:8080 --rm 785577973223.dkr.ecr.us-east-1.amazonaws.com/sagemaker-serving-containers/diffdl-container:latest`.

<h3>Prepare two models</h3>

We are going to deploy two different XGBoost models to our model server. We will need the serialized models and the inference scripts that we want to use.
We will store them in the current notebook folder, under <strong>model_and_code_1/</strong> and <strong>model_and_code_2/</strong>.

The purpose of using different models is to show that you can also deploy models that require diverse features and pre/post processing code.

First model is a regression model trained on the [Abalone data](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression.html) originally from the [UCI data repository](https://archive.ics.uci.edu/ml/datasets/abalone).
For further information, please refer to this [example](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/xgboost_abalone/xgboost_abalone.ipynb).

Second model is a binary classification model built by following this workshop: https://github.com/aws-samples/amazon-sagemaker-build-train-deploy

In [5]:
! rm -rf ./model_and_code_1/.ipynb_checkpoints
! rm -rf ./model_and_code_1/code/.ipynb_checkpoints
! rm -rf ./model_and_code_2/.ipynb_checkpoints
! rm -rf ./model_and_code_2/code/.ipynb_checkpoints

! tar -C ./model_and_code_1/ -cvzf model1.tar.gz ./
! tar -C ./model_and_code_2/ -cvzf model2.tar.gz ./

./
./xgboost-model
./code/
./code/predictor.py
./
./model.bin
./code/
./code/predictor.py


<h3>Deploy multiple models</h3>

In [6]:
model_data_prefix = 's3://{0}/{1}/modeldata'.format(bucket, prefix)

s3_model_1_path = model_data_prefix + '/model1.tar.gz'
!aws s3 cp model1.tar.gz {s3_model_1_path}
s3_model_2_path = model_data_prefix + '/model2.tar.gz'
!aws s3 cp model2.tar.gz {s3_model_2_path}

upload: ./model1.tar.gz to s3://sagemaker-us-east-1-785577973223/diffdl-container/modeldata/model1.tar.gz
upload: ./model2.tar.gz to s3://sagemaker-us-east-1-785577973223/diffdl-container/modeldata/model2.tar.gz


In [9]:
from time import gmtime, strftime
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.model import Model

model_name = 'multi-model-server-multidatamodel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model = Model(name = model_name,
              model_data = '',
              image_uri = container_image_uri,
              role=role,
              env = {
                  'SAGEMAKER_PROGRAM': 'predictor'
              },
              #predictor_cls = sagemaker.predictor.Predictor,
              sagemaker_session=sagemaker_session)

multi_model = MultiDataModel(name = model_name,
                             model_data_prefix = model_data_prefix,
                             model = model,
                             sagemaker_session=sagemaker_session)

<strong>Note:</strong> the environment variable SAGEMAKER_PREDICTOR is used to specify the name of the custom inference script.

In [ ]:
multi_endpoint_name = 'multi-model-server-ep-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
multi_endpoint_name = multi_endpoint_name + ('-gpu' if isGPU else '')
print(multi_endpoint_name)

# ml.m5.xlarge = 4vCPU/16GB; ml.g4dn.xlarge = 4vCPU/16GB.
instance = 'ml.g4dn.xlarge' if isGPU else 'ml.m5.xlarge'
print(instance)
# GPU instances can't be used for multi-model endpoints! 
model_to_deploy = model if isGPU else multi_model
pred = model_to_deploy.deploy(initial_instance_count=1,
                          instance_type=instance,
                          endpoint_name=multi_endpoint_name)

multi-model-server-ep-2021-07-20-12-02-45-gpu
ml.g4dn.xlarge
-------

<h3>Executing inferences</h3>

In [44]:
from IPython.display import Markdown as md
md("Go to `scripts` and execute `python3 test_endpoint.py -e {}`".format(multi_endpoint_name))

Go to `scripts` and execute `python3 test_endpoint.py -e multi-model-server-ep-2021-07-16-17-28-36`

--------------------------------------

Once the multi-model endpoint is ready, we can invoke either model1 or model2 by changing the target_model variable in the predict() function call.

In [9]:
from sagemaker.predictor import Predictor
pred = Predictor(multi_endpoint_name)
pred.serializer = sagemaker.serializers.CSVSerializer()

In [10]:
item = '77,33,143.0,101,212.2,102,104.9,120,15.3,4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1'
model_archive = '/model1.tar.gz'
pred.predict(item, target_model=model_archive)
#pred.predict(item)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "No module named 'xgboost'
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_inference/transformer.py", line 110, in transform
    self.validate_and_initialize(model_dir=model_dir)
  File "/usr/local/lib/python3.6/dist-packages/sagemaker_inference/transformer.py", line 158, in validate_and_initialize
    self._model = self._model_fn(model_dir)
  File "/opt/ml/models/e4c005b7fa0c40203fa9b3bbcf0b28cf/model/code/predictor.py", line 6, in model_fn
    model = pkl.load(open(model_file, 'rb'))
ModuleNotFoundError: No module named 'xgboost'
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/multi-model-server-ep-2021-07-16-14-25-28 in account 785577973223 for more information.

In [ ]:
item = '0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,73.0,79.0,32.0,27.0,45.0,48.0,13.0,62.0'
model_archive = '/model2.tar.gz'
pred.predict(item, target_model=model_archive)

In [ ]:
pred.delete_endpoint()
pred.delete_model()